In [1]:
import requests
import gzip
import pandas as pd
import networkx as nx
import numpy as np
import cPickle
import sys
from collections import Counter
import scipy.sparse
from scipy.sparse import coo_matrix, csc_matrix, csr_matrix, isspmatrix, issparse, isspmatrix_csr, isspmatrix_csc

%load_ext line_profiler
%load_ext autoreload
%autoreload 2

sys.path.insert(0, '/cellar/users/mikeyu/DeepTranslate/ddot')
import ddot
from ddot import Ontology

import pmc

import clixov_utils
import clixov_merge
import clique_maximal
import clique_maxcover
import mkl_spgemm
from clixov_merge import *
from clixov_utils import *

In [2]:
np.set_printoptions(linewidth=150)

In [3]:
# import warnings
# warnings.filterwarnings("error")

In [3]:
np.set_printoptions(precision=2)

In [5]:
# import os
# os.environ['NUMBA_DISABLE_JIT'] = '1'

# Load semantic similarity

## Yeast GO

In [4]:
f = np.load('yeast_go_20aug2017/resnik_ss.npz')
ss, ss_nodes = f['ss'], f['ss_nodes']

ont = Ontology.from_table('yeast_go_20aug2017/collapsed_go.20aug2017', clixo_format=True)

## Just the DNA repair branch of Yeast GO

In [6]:
f = np.load('/cellar/users/mikeyu/DeepCell/go_6oct2017_v2/resnik_ss_GO_0006281.npz')
ss, ss_nodes = f['ss'], f['ss_nodes']

ont = Ontology.from_table('/cellar/users/mikeyu/DeepCell/go_6oct2017_v2/collapsed_go_GO_0006281.propagated', clixo_format=True)

## Fan's HNeXO similarity

In [5]:
f = np.load('/cellar/users/mikeyu/DeepTranslate/hnexo/RFv2r3_square.npz')
ss, ss_nodes = f['rf'], f['genes']

# Iterate

In [5]:
ss = np.round(ss, 2)
np.triu_indices(ss.shape[0], k=1)
ss_flat_i, ss_flat_j = np.triu_indices(ss.shape[0], k=1)
ss_flat = ss[ss_flat_i, ss_flat_j]
print 'Distinct similarity scores:', np.unique(ss_flat).size

Distinct similarity scores: 408


In [6]:
## Initialize data structures
ss_argsort = np.argsort(-1 * ss_flat)

In [7]:
def count_unique_terms(ont, k):
    tmp = set([(tuple(sorted(ont.genes[g] for g in ont.term_2_gene[ont.terms[i]]))) for i in range(len(ont.terms)) if ont.term_sizes[i]==k])
    return len(tmp)

In [8]:
def collapse_identical(ont):
    ont = ont.propagate(gene_term=True, term_term=False, inplace=False)
    first = lambda x: x[0]
    import itertools
    term_groups = [[b for a, b in t_list] for _, t_list in itertools.groupby(sorted([(tuple(sorted(g)), t) for t, g in ont.term_2_gene.items()], key=first), key=first)]
    term_2_group = {t : '|'.join(t_list) for t_list in term_groups for t in t_list}
    return ont.rename(terms=term_2_group)

ont = collapse_identical(ont)

In [9]:
def expand_csr(X, new_shape):
    assert isspmatrix_csr(X)
    X.indptr = np.append(X.indptr, np.repeat(X.indptr[-1], new_shape[0] - X.shape[0]))
    X._shape = new_shape
    return X

In [16]:
def clixov(ss_argsort, n, beta, ont=None):
    dt = ss_flat[ss_argsort[0]]
    val_prev = dt
    dt_iter = 0
    n_genes = ss.shape[0]
    i_list, j_list = [], []

    n = ss.shape[0]
    G = csc_matrix((n,n), dtype=np.float32)
    X = np.zeros((n,n), dtype=np.float32)
    np.fill_diagonal(X, 1)
    X = csc_matrix(X)
    
    H = scipy.sparse.csr_matrix((n,n), dtype=np.bool)
    
    GX = csc_matrix((n,0), dtype=np.float32).toarray()
    prev_X_size = 0

    prev_merge = csr_matrix((n,n), dtype=np.int32)
#     prev_merge_i, prev_merge_j = np.array([], np.int32), np.array([], np.int32)
    
    if ont is not None:
        ont = ont.propagate(gene_term=True, term_term=False, inplace=False)
        term_sizes = np.array(ont.term_sizes)

    for ss_idx in ss_argsort:
        val = ss_flat[ss_idx]

        if val != val_prev or ss_idx==ss_argsort[-1]:
            prev_dt, dt, val_prev = dt, val_prev, val
            if ss_idx == ss_argsort[-1]:
                i, j = ss_flat_i[ss_idx], ss_flat_j[ss_idx]
                i_list.append(i)
                j_list.append(j)
            
            print '###########################################'
            start_clixov = time.time()
            expected_size = int(round(2**(-1 * dt) * n_genes))
            print 'dt_iter:', dt_iter, 'dt:', dt, 'prev_dt:', prev_dt, 'Expected size:', expected_size, round(2**(-1 * dt) * n_genes, 2)
            if ont is not None:
                print 'Ref terms:', (term_sizes==expected_size).sum()                
            print 'Total cliques:', X.shape[1], format_clique_sizes(X)

            dt_iter += 1
            print len(i_list), 'Added edges', G.sum() / 2, 'Current edges'

            # Only add edges that aren't in G yet
            tmp = as_dense_flat(G[i_list, j_list] == 0.)
            i_list, j_list = np.array(i_list)[tmp], np.array(j_list)[tmp]
            print "Edges not already in G:", i_list.size

            dG = coo_matrix((np.ones(i_list.size * 2, G.dtype),
                            (np.append(i_list, j_list), np.append(j_list, i_list))),
                            G.shape, copy=False)
            dG = csc_matrix(dG)
            newG = G + dG

            recent_X = X[:, prev_X_size:]
            recent_GX = as_dense_array(dot(newG, recent_X))
            GX = np.hstack([GX + as_dense_array(dot(dG, X[:,:prev_X_size])), recent_GX])
            
            # Reshape
            expand_csr(prev_merge, (X.shape[1], X.shape[1]))
            
#             if dt_iter==6:
#                 return X, GX, newG, G, dG, prev_merge_i, prev_merge_j, H
#             if (dt_iter % 10)==0:
#                 import cPickle
#                 with open('/cellar/users/mikeyu/Data/clixov/dt_iters2/%s.pkl' % dt_iter, 'wb') as f:
#                     cPickle.dump((X, GX, newG, beta, prev_merge, H, dG), f, protocol=cPickle.HIGHEST_PROTOCOL)
#     #                 np.savez('/cellar/users/mikeyu/Data/clixov/dt_iters/%s.npz' % dt_iter, X=X, GX=GX, newG=newG, beta=beta, prev_merge=prev_merge, H=H, dG=dG)

#             dX, merge_i, merge_j = calc_and_do_merge(X,
            dX, merge = calc_and_do_merge(X,
                                         X.toarray(),
                                         GX,
                                         newG,
                                         beta,
                                         prev_merge,
                                         H=H,
                                         dG=dG)
            assert_unique_cliques(dX)
            prev_merge += merge
            
#             prev_merge_i, prev_merge_j = np.append(prev_merge_i, merge_i), np.append(prev_merge_j, merge_j)
            print 'New cliques formed from merging:', dX.shape[1], format_clique_sizes(dX)
            
            unexplained = get_unexplained_edges(dX, dG)
            unexplained_n = unexplained.sum()
            print 'Unexplained edges:', unexplained_n
            assert unexplained_n == 0
            
            H = csr_matrix(update_subsumption(X, dX, H))
            assert np.all(H.data)
            
            prev_X_size = X.shape[1]
            G = newG

            if unexplained_n > 0:
                augment = clique_maxcover.max_clique_cover(unexplained, newG)
                augment = tuples_to_csc(augment, n, X.dtype)
                print 'Augmenting cliques:', augment.shape[1], format_clique_sizes(augment)
                assert augment.shape[1]>0
                X = csc_matrix(scipy.sparse.hstack([X, dX, augment]))
            else:
                X = csc_matrix(scipy.sparse.hstack([X, dX]))

            print 'prev_X_size:', prev_X_size        
            print 'prev_merge:', prev_merge.sum() / 2
            print 'Total iteration time:', time.time() - start_clixov
            i_list, j_list = [], []

        # Collect the gene pairs that have the same (or similar) similarities
    #     i, j = ss_idx / n_genes, ss_idx % n_genes
        i, j = ss_flat_i[ss_idx], ss_flat_j[ss_idx]
        i_list.append(i)
        j_list.append(j)
        
    return X

In [ ]:
n = ss.shape[0]
beta = 1.0
# X, GX, newG, G, dG, prev_merge_i, prev_merge_j, H = clixov(ss_argsort, n, beta, ont)
# X, GX, newG, G, dG, prev_merge_i, prev_merge_j, unexplained, dX, merge_i, merge_j, augment = clixov(ss_argsort, n, beta)
# X = clixov(ss_argsort, n, beta)

# X = clixov(ss_argsort, n, 1.0)
%lprun -f clixov -f calc_and_do_merge -f get_unexplained_edges -f get_merge_mask1 -f get_thresholds_symm -f subsumption -f do_merge -f get_unexplained_edges -f fill_diagonal X = clixov(ss_argsort, n, 1.0, ont)
# %lprun -f get_merge_mask1 X = clixov(ss_argsort, n, beta, ont)

###########################################
dt_iter: 0 dt: 11.65 prev_dt: 11.65 Expected size: 2 2.01
Ref terms: 675
Total cliques: 6448 OrderedDict([(1, 6448)])
675 Added edges 0.0 Current edges
Edges not already in G: 675
onlytest: 1350
Test 675 out of 41570256 clique pairs:
Test merge time: 0.0170359611511
To merge: 675
get_merge_mask1 time: 0.0204920768738
Iteration: 0 G/dG/H sum: 0 675 0 do_all: False
	Transferred from dG to G: 0
	Clique search tree nodes / time: 1857 0.0388679504395
	Meta cliques: 675 OrderedDict([(2, 675)])
Merge cliques (cover): 675 OrderedDict([(2, 675)])
	Unexplained meta edges: 0
Merge cliques: 675 0.000388860702515 OrderedDict([(2, 675)])
orig_dG_genes: 1350 0.0105929374695
Merge cliques (cover): 675 OrderedDict([(2, 675)])
do_merge time: 0.0787439346313
New cliques formed from merging: 675 OrderedDict([(2, 675)])
Unexplained edges: 0.0
prev_X_size: 6448
prev_merge: 675
Total iteration time: 0.249593019485
###########################################
dt_iter

onlytest: 39342
Test 19671 out of 71580060 clique pairs:
Test merge time: 0.0793099403381
To merge: 3899
get_merge_mask1 time: 0.0837500095367
Iteration: 0 G/dG/H sum: 18603 3899 8172 do_all: False
	Transferred from dG to G: 1292
	Clique search tree nodes / time: 1516 0.0747888088226
	Meta cliques: 200 OrderedDict([(2, 43), (3, 42), (4, 24), (5, 20), (6, 22), (7, 49)])
Merge cliques (cover): 200 OrderedDict([(3, 6), (4, 2), (5, 2), (6, 1), (7, 188), (8, 1)])
	Unexplained meta edges: 370
Iteration: 1 G/dG/H sum: 22317 185 8172 do_all: False
	Transferred from dG to G: 156
	Clique search tree nodes / time: 37 0.0401389598846
	Meta cliques: 0 OrderedDict()
Merge cliques (cover): 0 OrderedDict()
	Unexplained meta edges: 370
	*** No more new edges were explained ***
Iteration: 2 G/dG/H sum: 22317 185 8172 do_all: True
	Transferred from dG to G: 156
	Clique search tree nodes / time: 90 0.0104031562805
	Meta cliques: 19 OrderedDict([(2, 14), (3, 5)])
Merge cliques (cover): 19 OrderedDict([(3, 

	Clique search tree nodes / time: 1313 0.10928106308
	Meta cliques: 130 OrderedDict([(2, 19), (3, 21), (4, 17), (5, 15), (6, 14), (7, 9), (8, 5), (9, 7), (10, 8), (11, 15)])
Merge cliques (cover): 130 OrderedDict([(3, 5), (4, 4), (5, 4), (6, 4), (7, 2), (11, 111)])
	Unexplained meta edges: 862
Iteration: 1 G/dG/H sum: 49848 431 14981 do_all: False
	Transferred from dG to G: 286
	Clique search tree nodes / time: 112 0.0752038955688
	Meta cliques: 2 OrderedDict([(2, 1), (4, 1)])
Merge cliques (cover): 2 OrderedDict([(4, 1), (11, 1)])
	Unexplained meta edges: 736
Iteration: 2 G/dG/H sum: 49911 368 14981 do_all: False
	Transferred from dG to G: 216
	Clique search tree nodes / time: 106 0.0691180229187
	Meta cliques: 0 OrderedDict()
Merge cliques (cover): 0 OrderedDict()
	Unexplained meta edges: 736
	*** No more new edges were explained ***
Iteration: 3 G/dG/H sum: 49911 368 14981 do_all: True
	Transferred from dG to G: 216
	Clique search tree nodes / time: 395 0.0190920829773
	Meta cliques

Total iteration time: 0.825762033463
###########################################
dt_iter: 13 dt: 8.75 prev_dt: 8.85 Expected size: 15 14.98
Ref terms: 71
Total cliques: 9501 OrderedDict([(1, 6448), (2, 675), (3, 469), (4, 366), (5, 266), (6, 235), (7, 198), (8, 170), (9, 145), (10, 125), (11, 122), (12, 108), (13, 97), (14, 76), (15, 1)])
3939 Added edges 35232.0 Current edges
Edges not already in G: 3939
onlytest: 161474
Test 80737 out of 90259500 clique pairs:
Test merge time: 0.14265203476
To merge: 6907
get_merge_mask1 time: 0.149685144424
Iteration: 0 G/dG/H sum: 75271 6907 21393 do_all: False
	Transferred from dG to G: 4994
	Clique search tree nodes / time: 773 0.119755983353
	Meta cliques: 70 OrderedDict([(2, 15), (3, 13), (4, 5), (5, 5), (6, 5), (7, 5), (8, 4), (9, 2), (10, 3), (11, 3), (12, 1), (13, 2), (14, 5), (15, 2)])
Merge cliques (cover): 70 OrderedDict([(4, 1), (5, 2), (6, 1), (7, 2), (15, 64)])
	Unexplained meta edges: 758
Iteration: 1 G/dG/H sum: 81799 379 21393 do_al

	Clique search tree nodes / time: 759 0.0295639038086
	Meta cliques: 112 OrderedDict([(2, 26), (3, 21), (4, 27), (5, 18), (6, 8), (7, 3), (8, 3), (9, 1), (11, 1), (13, 1), (14, 3)])
Merge cliques (cover): 112 OrderedDict([(3, 2), (4, 5), (5, 16), (6, 21), (7, 15), (8, 9), (9, 7), (10, 7), (11, 2), (12, 3), (14, 3), (15, 4), (16, 5), (17, 2), (18, 11)])
	Unexplained meta edges: 2
Iteration: 3 G/dG/H sum: 109578 1 25870 do_all: True
	Transferred from dG to G: 0
	Clique search tree nodes / time: 4 0.021852016449
	Meta cliques: 1 OrderedDict([(3, 1)])
Merge cliques (cover): 1 OrderedDict([(5, 1)])
	Unexplained meta edges: 0
Merge cliques: 181 0.000504016876221 OrderedDict([(3, 4), (4, 7), (5, 17), (6, 22), (7, 16), (8, 10), (9, 7), (10, 7), (11, 4), (12, 4), (14, 4), (15, 4), (16, 8), (17, 6), (18, 61)])
orig_dG_genes: 8902 0.0522789955139
Merge cliques (cover): 63 OrderedDict([(14, 1), (16, 1), (18, 61)])
do_merge time: 0.4748442173
New cliques formed from merging: 63 OrderedDict([(14, 1)

...

	Clique search tree nodes / time: 293 0.444829940796
	Meta cliques: 0 OrderedDict()
Merge cliques (cover): 0 OrderedDict()
	Unexplained meta edges: 14958
	*** No more new edges were explained ***
Iteration: 2 G/dG/H sum: 1720872 7479 119913 do_all: True
	Transferred from dG to G: 6986
	Clique search tree nodes / time: 7572417 33.1749298573
	Meta cliques: 1495238 OrderedDict([(2, 3), (3, 9), (4, 7), (5, 33), (6, 73), (7, 232), (8, 493), (9, 1081), (10, 2201), (11, 4086), (12, 6022), (13, 7450), (14, 8753), (15, 9845), (16, 11323), (17, 12435), (18, 13799), (19, 15914), (20, 19570), (21, 24875), (22, 33133), (23, 43423), (24, 57076), (25, 68394), (26, 77737), (27, 86048), (28, 92144), (29, 96778), (30, 97934), (31, 93805), (32, 83719), (33, 71275), (34, 59998), (35, 50257), (36, 44405), (37, 39400), (38, 34214), (39, 28798), (40, 24536), (41, 21558), (42, 19237), (43, 17069), (44, 15432), (45, 13359), (46, 11695), (47, 10807), (48, 10213), (49, 9935), (50, 8350), (51, 7255), (52, 6407), 

Merge cliques (cover): 62706 OrderedDict([(7, 1), (8, 9), (9, 13), (10, 53), (11, 80), (12, 269), (13, 582), (14, 872), (15, 1157), (16, 1730), (17, 2156), (18, 2968), (19, 3206), (20, 3306), (21, 2925), (22, 2690), (23, 2286), (24, 1895), (25, 1925), (26, 2013), (27, 2226), (28, 2274), (29, 2406), (30, 2485), (31, 2798), (32, 2528), (33, 2541), (34, 2417), (35, 2188), (36, 1841), (37, 1299), (38, 1055), (39, 786), (40, 676), (41, 490), (42, 488), (43, 362), (44, 410), (45, 369), (46, 321), (47, 324), (48, 275), (49, 291), (50, 283), (51, 202), (52, 219), (53, 154), (54, 129), (55, 107), (56, 87), (57, 69), (58, 43), (59, 45), (60, 54), (61, 45), (62, 23), (63, 41), (64, 27), (65, 30), (66, 18), (67, 18), (68, 14), (69, 9), (70, 9), (71, 4), (72, 2), (73, 13), (74, 9), (75, 19), (76, 8), (77, 4), (78, 3), (79, 3), (80, 3), (81, 3), (82, 2), (83, 2), (84, 3), (85, 5), (86, 4), (87, 1), (89, 1), (90, 1), (133, 1), (134, 3)])
	Unexplained meta edges: 14
Iteration: 3 G/dG/H sum: 1732100 7 

Merge cliques: 386590 0.378588914871 OrderedDict([(5, 1), (6, 2), (7, 11), (8, 44), (9, 101), (10, 187), (11, 259), (12, 432), (13, 443), (14, 643), (15, 720), (16, 997), (17, 1533), (18, 2062), (19, 2945), (20, 3170), (21, 3493), (22, 3791), (23, 3705), (24, 3940), (25, 3899), (26, 4105), (27, 4386), (28, 4229), (29, 4319), (30, 4269), (31, 4378), (32, 4689), (33, 4767), (34, 5230), (35, 6351), (36, 7814), (37, 9335), (38, 10588), (39, 11711), (40, 13013), (41, 13224), (42, 13829), (43, 13239), (44, 13397), (45, 12716), (46, 11837), (47, 11019), (48, 10921), (49, 9940), (50, 8944), (51, 9072), (52, 8147), (53, 7197), (54, 6853), (55, 6626), (56, 6345), (57, 6401), (58, 6176), (59, 5720), (60, 5632), (61, 5473), (62, 4933), (63, 4756), (64, 4433), (65, 4255), (66, 4175), (67, 3874), (68, 3605), (69, 3216), (70, 3096), (71, 2906), (72, 2971), (73, 2960), (74, 2645), (75, 2139), (76, 1703), (77, 1424), (78, 1198), (79, 908), (80, 891), (81, 1027), (82, 816), (83, 797), (84, 703), (85, 52

	Unexplained meta edges: 0
Merge cliques: 10433190 7.0758960247 OrderedDict([(4, 1), (5, 2), (6, 6), (7, 42), (8, 117), (9, 282), (10, 691), (11, 1589), (12, 2884), (13, 5583), (14, 9668), (15, 16757), (16, 26246), (17, 39409), (18, 56621), (19, 76170), (20, 100707), (21, 126497), (22, 158477), (23, 195744), (24, 236653), (25, 281661), (26, 328692), (27, 381384), (28, 436827), (29, 496930), (30, 548303), (31, 584281), (32, 600250), (33, 606004), (34, 599527), (35, 585296), (36, 561980), (37, 518467), (38, 459100), (39, 401356), (40, 342297), (41, 287313), (42, 237837), (43, 198417), (44, 162991), (45, 133441), (46, 107906), (47, 88813), (48, 74814), (49, 62637), (50, 51486), (51, 42424), (52, 34845), (53, 28661), (54, 23403), (55, 18962), (56, 15197), (57, 12292), (58, 10780), (59, 8312), (60, 6691), (61, 5744), (62, 4652), (63, 4118), (64, 3259), (65, 3133), (66, 2714), (67, 2473), (68, 1901), (69, 1671), (70, 1583), (71, 1234), (72, 1023), (73, 864), (74, 791), (75, 517), (76, 508), 

TypeError: UPDATEIFCOPY used for non-array input.

In [14]:
with open('tmp.npy') as f:
    X, dG = cPickle.load(f)

In [ ]:
import networkx as nx
import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout

In [96]:
# H = csr_matrix(subsumption(X))
# # H = csr_matrix(subsumption(X[:,[7, 6448]]))
# import igraph
# g = igraph.Graph(n=H.shape[0], edges=zip(*H.nonzero()), directed=True)
# assert g.is_dag()
# topo = g.topological_sorting(mode='out')
# print [(e.source, e.target) for e in g.es[:5]]
# # print list(topo)
# print topo.index(7), topo.index(6448)

# # import clique_maximal
# # clique_maximal.BK_hier_Gsep_wrapper(Gold, Gnew, H)

In [16]:
A, GA, G = X.toarray(), GX, newG
# A, GA, G = X.toarray()[:,[345, 703]], GX[:,[345, 703]], newG

threshold_loose, threshold_strict = get_thresholds(A, A, beta)
i_merge, j_merge = get_merge_mask1(A, GA, A, GA, G,
                                   threshold_loose, threshold_strict,
                                   symm=True,
                                   notest=(prev_merge_i, prev_merge_j))
# i_merge, j_merge = get_merge_mask1(A, GA, A, GA, G,
#                                    threshold_loose, threshold_strict,
#                                    symm=True)

Test 1044336 out of 25365003 clique pairs:
Merge time: 1.99702906609
To merge: 2755


In [ ]:
merge = do_merge(X, i_merge, j_merge, prev_merge_i, prev_merge_j)

# max_size = 0
# for i in range(merge.shape[1]):
#     clique = tuple(sorted(merge[:,i].nonzero()[0]))
#     max_size = max(max_size, len(clique))
#     if len(clique)>=4:
#         print '*', as_dense_array(G[clique,:][:,clique])
#     assert_clique(clique, G)
# print 'max_size:', max_size

In [54]:
newX, merge_i, merge_j = calc_and_do_merge(X,
                                           X.toarray(),
                                           GX,
                                           newG,
                                           beta,
                                           prev_merge_i,
                                           prev_merge_j,
                                           H=H,
                                           dG=dG)

onlytest: 416
Test 197 out of 106602 clique pairs:
Test merge time: 0.000344038009644
To merge: 113
get_merge_mask1 time: 0.00427007675171
Iteration: 0 G/dG sum: 448 226
[232 286]
[ 15 170 232]
	Transferred from dG to G: 60
	Search tree nodes / time: 74 0.0268878936768
	Meta cliques: 3 OrderedDict([(5, 2), (6, 1)])
	Unexplained meta edges: 0
Merge cliques: 3 0.000420093536377 OrderedDict([(8, 3)])
orig_dG_genes: 116 0.00335502624512
Merge cliques (cover): 3 OrderedDict([(8, 3)])
do_merge time: 0.0528020858765


# asdf

In [ ]:
with open('tmp.npy', 'wb') as f: cPickle.dump((Gold, Gnew, H), f)

In [102]:
np.all(subsumption(X).toarray() == H.toarray())

False

In [111]:
print H[310,:].nonzero()[1]
print H[315,:].nonzero()[1]

[]
[]


In [14]:
with open('tmp.npy') as f:
    Gold, Gnew, H = cPickle.load(f)

In [75]:
C, CP, CN, tree = clique_maximal.BK_dG_py(Gold, Gnew)
cliques_csc = cliques_to_csc(C, CP, CN, n)
print cliques_csc.shape
for i in range(cliques_csc.shape[1]):
    print cliques_csc[:,i].nonzero()[0]

(304, 3)
[  0 104 136 153 221 232 257 286]
[ 17 128 138 148 231 233 249 270]
[ 17  54 147 162 189 196 245 274]


In [76]:
[g for t, g in ont.term_2_gene.items() if len(g)==8]

[[0, 104, 136, 153, 221, 232, 257, 286],
 [17, 128, 138, 148, 231, 233, 249, 270],
 [17, 54, 147, 162, 189, 196, 245, 274]]

In [ ]:
C, CP, CN, tree = clique_maximal.BK_hier_dG_py(Gold, Gnew, H)
cliques_csc = cliques_to_csc(C, CP, CN, n)
print cliques_csc.shape

In [74]:
Gold, Gnew = clixov_utils.get_test_new_network('cluster', k=200, r=10, s=10, verbose=False)
%time cliques, cliques_indptr, cliques_n, tree_size = clique_maximal.BKPivotSparse2_Gnew_wrapper(Gold, Gnew)
cliques = cliques_to_csc(cliques, cliques_indptr, cliques_n, Gold.shape[0])

tmp1 = csc_to_cliques_list(cliques)
%time tmp2 = clique_maximal.get_cliques_igraph(Gold.shape[0], Gold + Gnew, Gnew, input_fmt='matrix')
assert set(tmp1) == set(tmp2)
len(tmp1)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2.45 ms
CPU times: user 152 ms, sys: 0 ns, total: 152 ms
Wall time: 149 ms


37

In [145]:
X_set = set([])
for i in as_dense_flat(X.sum(0)==4).nonzero()[0]:
    X_set.add(tuple(sorted(X[:, i].nonzero()[0])))
len(X_set)

386

In [144]:
ont_set = set([tuple(sorted(ont.term_2_gene[t])) for t, s in zip(ont.terms, ont.term_sizes) if s==4])
len(ont_set)

303